In [2]:
import sys
import os
import re
import numpy as np
import pandas as pd

import mne
from mne_bids import write_raw_bids, BIDSPath


In [ ]:
# set different path to load and save data
ds_root='../data'
tgt_dir='..'

dir_preprocess=os.path.join(tgt_dir,'data','preprocessData')
if not os.path.exists(dir_preprocess):
    os.makedirs(dir_preprocess)


In [4]:
# BIDS format file name part labels
BIDS_fn_label = []
BIDS_fn_label.append('_task-dynamic')      # BIDS file name task label. format: [_task-<task_label>]
BIDS_fn_label.append('_eeg')       

In [ ]:
# preprocess EEG data
sub_dir=[f'sub-STSWD{i}' for i in range(1117,1119)]
for sub in sub_dir:
    raw=mne.io.read_raw_brainvision(os.path.join(ds_root,sub,'eeg',sub+BIDS_fn_label[0]+BIDS_fn_label[1]+'.vhdr'))
       
    raw.set_channel_types({'ECG':'ecg',
                           'HEOGL': 'eog',  
                           'HEOGR': 'eog'})
    
    # resample
    raw.resample(500, npad="auto")
    # filter
    raw.filter(1, 30, fir_design='firwin', picks=['eeg'])  

    # re-reference
    raw.set_eeg_reference('average')

    # ica remove artifact
    ica = mne.preprocessing.ICA(n_components=50, random_state=97)
    ica.fit(raw) 
    ica.exclude = []                                   
    eog_indices, eog_scores = ica.find_bads_eog(raw)                                                         
    ecg_indices, ecg_scores = ica.find_bads_ecg(raw, method='ctps')               
    ica.exclude = eog_indices + ecg_indices 
    ica.apply(raw) 

    # save data as bids
    bids_root = dir_preprocess
    bids_path = BIDSPath(
        subject=sub.replace('sub-',''),  # BIDS can not identify '-'
        task='MAAT',  #  Multi-Attribute Attention Task 
        datatype='eeg',
        root=bids_root
    )

    # 保存到 BIDS 格式
    write_raw_bids(
        raw=raw,
        bids_path=bids_path,
        overwrite=True,
        allow_preload=True,
        format='BrainVision'  # 保持 BrainVision 格式（.vhdr/.vmrk/.eeg）
    )

Extracting parameters from ../data/sub-STSWD1117/eeg/sub-STSWD1117_task-dynamic_eeg.vhdr...
Setting channel info structure...


/var/folders/xc/16fqwx1n1gbfb77yd1bdg8mh0000gn/T/ipykernel_79878/3415862983.py:3: RuntimeWarning: Online software filter detected. Using software filter settings and ignoring hardware values
  raw=mne.io.read_raw_brainvision(os.path.join(ds_root,sub,'eeg',sub+BIDS_fn_label[0]+BIDS_fn_label[1]+'.vhdr'))
/var/folders/xc/16fqwx1n1gbfb77yd1bdg8mh0000gn/T/ipykernel_79878/3415862983.py:3: RuntimeWarning: Channels contain different highpass filters. Lowest (weakest) filter setting (0.00 Hz) will be stored.
  raw=mne.io.read_raw_brainvision(os.path.join(ds_root,sub,'eeg',sub+BIDS_fn_label[0]+BIDS_fn_label[1]+'.vhdr'))
/var/folders/xc/16fqwx1n1gbfb77yd1bdg8mh0000gn/T/ipykernel_79878/3415862983.py:3: RuntimeWarning: Channels contain different lowpass filters. Highest (weakest) filter setting (500.00 Hz, Nyquist limit) will be stored.
  raw=mne.io.read_raw_brainvision(os.path.join(ds_root,sub,'eeg',sub+BIDS_fn_label[0]+BIDS_fn_label[1]+'.vhdr'))


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 1651 samples (3.302 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.3s


EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.


...

Using threshold: 0.23 for CTPS ECG detection
Using channel ECG to identify heart beats.
Setting up band-pass filter from 8 - 16 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 8.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 7.75 Hz)
- Upper passband edge: 16.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 16.25 Hz)
- Filter length: 5000 samples (10.000 s)

Number of ECG events detected : 4846 (average pulse 75.91169221772005 / min.)
Not setting metadata
4846 matching events found
No baseline correction applied
Using data from preloaded Raw for 4846 events and 501 original time points ...
1 bad epochs dropped
Applying ICA to Raw instance
    Transforming to ICA space (50 components)
    Zeroing out 4 ICA components
    Projecting back using 62 PCA components


/Users/hyijie/Desktop/Lab/CPP_Multiverse_stage2/.venv/lib/python3.13/site-packages/mne/preprocessing/ica.py:2401: RuntimeWarning: divide by zero encountered in matmul
  mixing = pca_components.T @ mixing
/Users/hyijie/Desktop/Lab/CPP_Multiverse_stage2/.venv/lib/python3.13/site-packages/mne/preprocessing/ica.py:2401: RuntimeWarning: overflow encountered in matmul
  mixing = pca_components.T @ mixing
/Users/hyijie/Desktop/Lab/CPP_Multiverse_stage2/.venv/lib/python3.13/site-packages/mne/preprocessing/ica.py:2401: RuntimeWarning: invalid value encountered in matmul
  mixing = pca_components.T @ mixing


Writing '../data/preprocessData/participants.tsv'...
Writing '../data/preprocessData/participants.json'...
The provided raw data contains annotations, but you did not pass an "event_id" mapping from annotation descriptions to event codes. We will generate arbitrary event codes. To specify custom event codes, please pass "event_id".
Used Annotations descriptions: [np.str_('Comment/actiCAP not connected'), np.str_('New Segment/'), np.str_('Stimulus/S  1'), np.str_('Stimulus/S  2'), np.str_('Stimulus/S  6'), np.str_('Stimulus/S  8'), np.str_('Stimulus/S 16'), np.str_('Stimulus/S 17'), np.str_('Stimulus/S 20'), np.str_('Stimulus/S 24'), np.str_('Stimulus/S 64'), np.str_('Stimulus/S 72'), np.str_('Stimulus/S 96'), np.str_('Stimulus/S128'), np.str_('Stimulus/S130'), np.str_('Stimulus/S255')]
Writing '../data/preprocessData/sub-STSWD1118/eeg/sub-STSWD1118_task-MAAT_events.tsv'...
Writing '../data/preprocessData/sub-STSWD1118/eeg/sub-STSWD1118_task-MAAT_events.json'...
Writing '../data/preproc

/var/folders/xc/16fqwx1n1gbfb77yd1bdg8mh0000gn/T/ipykernel_79878/3415862983.py:36: RuntimeWarning: Converting data files to BrainVision format
  write_raw_bids(
/var/folders/xc/16fqwx1n1gbfb77yd1bdg8mh0000gn/T/ipykernel_79878/3415862983.py:36: RuntimeWarning: Encountered data in "short" format. Converting to float32.
  write_raw_bids(


Writing '../data/preprocessData/sub-STSWD1118/sub-STSWD1118_scans.tsv'...
Wrote ../data/preprocessData/sub-STSWD1118/sub-STSWD1118_scans.tsv entry with eeg/sub-STSWD1118_task-MAAT_eeg.vhdr.
